In [1]:
# connexion avec la base de données
#from pymongo import MongoClient
# import sqlite3

# traitement des données
# import json
import pandas as pd
# import re
# import os

# graphiques
# import matplotlib.pyplot as plt
# import seaborn as sns
# import plotly.express as px

import matplotlib
matplotlib.use('TkAgg') # o 'Qt5Agg' # Esto es solo para ver los graficos en Vscode


# nuage de mots
# import numpy as np
# from PIL import Image

# Preprocessing
# from sklearn.impute import SimpleImputer
# from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder #, OneHotEncoder, StandardScaler, RobustScaler, MinMaxScaler, FunctionTransformer


from sklearn.feature_extraction.text import TfidfVectorizer

# Pipeline and model
# from sklearn.pipeline import Pipeline
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import GridSearchCV

# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import CategoricalNB, GaussianNB, BernoulliNB, MultinomialNB
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.svm import SVC
# from sklearn.svm import LinearSVC
# from sklearn.tree import ExtraTreeClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn import svm
# from sklearn.neighbors import KNeighborsClassifier 

# Score of models
#from sklearn.metrics import plot_confusion_matrix
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import classification_report, roc_curve, auc
# from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

# Save models
import pickle

# Regression test
import unittest

In [2]:
data_news_sms = pd.read_csv('SMSSpam2', sep='\t',header=None, names=['type', 'text'])

data_news_sms

,type,text
0,ham,Mom says you look good with that haircut.
1,ham,See you at the restaurant
2,ham,The keys are on the table
3,spam,"IMPORTANT - You could be entitled up to £3,160..."
4,spam,"You've won a prize, a gift card, or a coupon t..."
5,spam,You’re being offered a low or no interest cred...
6,spam,You have been overcharged and are owed a refun...
7,spam,A [redacted] loan for £950 is approved for you...
8,spam,You can get help to pay off your student debt ...
9,spam,There is negative information in your credit r...


In [3]:
data = pd.read_csv('spam.csv',encoding = "latin-1")
# data = pd.read_csv('authentification/Projet_spam/spam.csv',encoding = "latin-1")

# Suppression of unnecessary columns
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'], axis=1)
data = data.rename(columns={'v1': 'type', 'v2' : 'text'})

#Suppression of duplicate lines
data = data.drop_duplicates()

In [4]:
target = 'type'

In [5]:
# # Finalisation du modèle
# # Entraînement avec toutes les données

# # CV - best parameters: {'model__alpha': 0.1, 'model__fit_prior': True}

# def finalisation_du_modele(df):
#     # target preprocessing
#     lb_encod = LabelEncoder()
#     y = lb_encod.fit_transform(df['type'])
    
#     # TFIDF
#     tfidf= TfidfVectorizer(max_features=3000)
#     X=tfidf.fit_transform(df['text']).toarray()
    
#     # # Scaler
#     # transformer = FunctionTransformer(np.log1p, validate=True)
#     # X = transformer.transform(X)
    
#     clf_MNB = MultinomialNB(alpha=0.1, fit_prior=True)

#     clf_MNB.fit(X, y)
    
#     return clf_MNB

In [6]:
# modele_final_MNB = finalisation_du_modele(df=data)

In [7]:
# Récupération du modèle qui a donné les meilleurs résultats
loaded_model_MNB = pickle.load(open('trained_model_MultinomialNB().pkl', 'rb'))

# loaded_model_MNB = pickle.load(open(r'authentification\Projet_spam\trained_model_MultinomialNB().pkl', 'rb'))

In [8]:
data_news_sms

,type,text
0,ham,Mom says you look good with that haircut.
1,ham,See you at the restaurant
2,ham,The keys are on the table
3,spam,"IMPORTANT - You could be entitled up to £3,160..."
4,spam,"You've won a prize, a gift card, or a coupon t..."
5,spam,You’re being offered a low or no interest cred...
6,spam,You have been overcharged and are owed a refun...
7,spam,A [redacted] loan for £950 is approved for you...
8,spam,You can get help to pay off your student debt ...
9,spam,There is negative information in your credit r...


In [9]:
# df=data_news_sms, target=target, model=loaded_model_MN

def Pred_news_spams(df, target, model):

    # target preprocessing
    lb_encod = LabelEncoder()
    y = lb_encod.fit_transform(df[target])
    # ham = 0, spam = 1 

    #Traitement des messages avec TFIDF
    tfidf= TfidfVectorizer(max_features=3000)
    tfidf.fit(data['text'])

    # Dans ce cas, au lieu d'utiliser .fit_transform, nous utilisons simplement .transform 
    # et de cette manière, nous créons le TFIDF avec TfidfVectorizer qui a déjà été entraîné avec toutes les données.
    X_news=tfidf.transform(df['text']).toarray()

    # Nous faisons les prédictions avec notre modèle
    # ham = 0, spam = 1 
    predection = model.predict(X_news)

    # Nous faisons le processus inverse de label encoder 
    # pour trouver le nom de nos prédictions.
    predection_nom = lb_encod.inverse_transform(predection)

    # on trouve la probabilité de la prédiction
    proba = model.predict_proba(X_news)

    # nous ajoutons ces nouvelles informations à notre ensemble de données original
    df['prediction'] = predection_nom
    df['probabilité_spam (%)'] = proba[:,1]*100
    df['probabilité_spam (%)'] = round(df['probabilité_spam (%)'], 1)

    return df

In [10]:
data_news_sms_pred = Pred_news_spams(df=data_news_sms, target=target, model=loaded_model_MNB)

In [11]:
data_news_sms_pred

,type,text,prediction,probabilité_spam (%)
0,ham,Mom says you look good with that haircut.,ham,0.7
1,ham,See you at the restaurant,ham,3.4
2,ham,The keys are on the table,ham,5.9
3,spam,"IMPORTANT - You could be entitled up to £3,160...",spam,91.9
4,spam,"You've won a prize, a gift card, or a coupon t...",spam,65.2
5,spam,You’re being offered a low or no interest cred...,spam,56.5
6,spam,You have been overcharged and are owed a refun...,ham,14.4
7,spam,A [redacted] loan for £950 is approved for you...,spam,98.8
8,spam,You can get help to pay off your student debt ...,ham,36.5
9,spam,There is negative information in your credit r...,ham,19.3


In [1]:
import pandas as pd

# Crear un DataFrame de ejemplo
data = {'Columna': ['Hola', 'Mundo', 'OpenAI', 'GPT-3', 'Python']}
df = pd.DataFrame(data)

# Verificar la longitud de las líneas en la columna 'Columna'
df['Longitud mayor a 5'] = df['Columna'].str.len() > 5

# Imprimir el DataFrame resultante
print(df)

  Columna  Longitud mayor a 5
0    Hola               False
1   Mundo               False
2  OpenAI                True
3   GPT-3               False
4  Python                True


In [4]:
data = {'Columna': ['Hola', 'Mundo', 'OpenAI', 'GPT-3', 'Python']}
df = pd.DataFrame(data)
any(df['Columna'].str.len() > 5)

True